In [1]:
import ipywidgets as widgets
import pyvista
import numpy as np
from IPython.display import display, clear_output
from tvbwidgets.ui.connectivity_matrix_editor_widget import ConnectivityMatrixEditor
from tvbwidgets.api import ConnectivityWidget
from tvb.datatypes.connectivity import Connectivity

# 1. Load connectivity
conn = Connectivity().from_file()
conn.configure()

# 2. Create custom widgets
class CustomMatrixEditor(widgets.VBox, ConnectivityMatrixEditor):
    def __init__(self, connectivity, viewer, **kwargs):
        ConnectivityMatrixEditor.__init__(self, connectivity, **kwargs)  # Initialize TVB Widget
        widgets.VBox.__init__(self)  # Initialize ipywidget
        self.viewer = viewer
        self.output = widgets.Output()
        
        with self.output:
            self.display()  # TVB Widget needs explicit display()
        
        self.children = [self.output]  # Add wrapped output to VBox

    def on_cell_clicked(self, x, y, matrix_name):
        super().on_cell_clicked(x, y, matrix_name)
        if hasattr(self.viewer, "highlight_centres"):
            self.viewer.highlight_centres(self.selected_centres)

class CustomViewer(ConnectivityWidget):  
    def __init__(self, connectivity, width=600, height=600):
        super().__init__(connectivity, default_active_tab='viewers', width=width, height=height)
        self.highlight_output = widgets.Output()  # Output widget to show highlights
        self.children = list(self.children) + [self.highlight_output]  # Add it to children
    
    def highlight_centres(self, selected_centres):
        with self.highlight_output: 
            clear_output(wait=True)
            connectivity_3d_viewer = self.viewers_tab.children[1]
            if hasattr(connectivity_3d_viewer, "output"):
              plotter = connectivity_3d_viewer.output.plotter
              if hasattr(self,"highlight_actor") and self.highlight_actor: 
                  plotter.remove_actor(self.highlight_actor, reset_camera=False)
              points = np.array(selected_centres)
              mesh_points=pyvista.PolyData(points)  
              points_size=connectivity_3d_viewer.output.CONFIG.point_size + 5
              self.highlight_actor=plotter.add_points(mesh_points,color="red",point_size=points_size)
              connectivity_3d_viewer.output.update_plot()

    def custom_display(self):
        display(self)  # Display the inherited widget

# 3. Initialize Widgets
custom_viewer = CustomViewer(conn, width=600, height=600)
custom_matrix_editor = CustomMatrixEditor(conn, viewer=custom_viewer)

# 4. Display both widgets side by side
side_by_side = widgets.HBox(
    [custom_matrix_editor, custom_viewer],  
    layout=widgets.Layout(
        width='100%',
        justify_content='space-between',
        padding='10px',
        border='1px solid #eee'
    )
)

display(side_by_side)


02-04-2025 08:13:45 - INFO - tvbwidgets - Version: 2.2.1
Using matplotlib as 2D backend.
2025-04-02 20:14:11,851 - INFO - tvb.storage.h5.encryption.data_encryption_handler - Cannot import syncrypto library.
02-04-2025 08:14:12 - INFO - tvbwidgets.core.pse.parameters - ImportError: Dask dependency is not included, so this functionality won't be available
2025-04-02 20:14:13,020 - WARNING - tvb.basic.readers - File 'hemispheres' not found in ZIP.
